In [1]:
import numpy as np
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, MultiPolygon, Point

d1 = np.array([[0,0], [0,1],  [1,0], [1,1]])
d2 = d1 + 0.5
d3 = d2 + 0.5
h1 = ConvexHull(d1)
h2 = ConvexHull(d2)
h3 = ConvexHull(d3)
v1 = [d1[v] for v in h1.vertices]
v2 = [d2[v] for v in h2.vertices]
v3 = [d3[v] for v in h3.vertices]
s1 = Polygon(v1)
s2 = Polygon(v2)
s3 = Polygon(v3)
polygons = [s1, s2, s3]
print reduce(lambda a,b: a.union(b), polygons[1:], polygons[0]).area
p = [Point(c) for c in s1.exterior.coords]
max([p1.distance(p2) for p1 in p for p2 in p])

2.5


1.4142135623730951

In [10]:
def clusterGeometryMetrics(drum_points, labels):
    
    grouped_drum_points = {}
    
    for i in range(len(drum_points)):
        if labels[i] not in grouped_drum_points:
            grouped_drum_points[labels[i]] = list()
        grouped_drum_points[labels[i]].append(drum_points[i])
    
    num_labels = len(grouped_drum_points)
    drum_polygons = {}
    
    for drum in grouped_drum_points:
        points = grouped_drum_points[drum]
        hull = ConvexHull(grouped_drum_points[drum])
        drum_polygons[drum] = Polygon([points[v] for v in hull.vertices])
    
    total_hull = ConvexHull(drum_points)
    total_polygon = Polygon([drum_points[v] for v in total_hull.vertices])
    p = [Point(c) for c in total_polygon.exterior.coords]
    total_diameter = max([p1.distance(p2) for p1 in p for p2 in p])
    
    calc_intersect = lambda i, j: drum_polygons[i].intersection(drum_polygons[j]).area / total_polygon.area
    calc_roundness = lambda poly : 4 * np.pi * poly.area / (poly.length**2)
    calc_distance = lambda i, j: drum_polygons[i].distance(drum_polygons[j]) / total_diameter
    
    #overlap areas between the polygons of the different clusters, normalized by area of total plot polygon
    pairwise_intersect_areas = [[calc_intersect(i,j) for i in range(num_labels)] for j in range(num_labels)]
    
    
    # Distance between nearest points of 2 polygons normalized by "diameter" of total plot polygon
    pairwise_distances = [[calc_distance(i,j) for i in range(num_labels)] for j in range(num_labels)]
    
    #measure of "roundness" of each polygon based on Polsby-Popper Test
    roundness = [calc_roundness(drum_polygons[d]) for d in drum_polygons]
    
    #relative sizes of each cluster polygon normalized by area of total plot polygon
    relative_areas = [drum_polygons[d].area / total_polygon.area for d in drum_polygons]
    
    #the polygons themselves for further processing 
    print type(drum_polygons.values())
    polygons = drum_polygons.values() + [total_polygon]
    
    #the area of the union of all cluser polygons over the area of the total plot
    overlap_ratio = reduce(lambda a,b: a.union(b), polygons[1:], polygons[0]).area / sum([p.area for p in polygons])
    
    returnVal = {}
    returnVal['pairwise_intersect_areas'] = pairwise_intersect_areas
    returnVal['roundness'] = roundness
    returnVal['relative_areas'] = relative_areas
    returnVal['overlap_ratio'] = overlap_ratio
    returnVal['polygons'] = polygons
    returnVal['pairwise_distances'] = pairwise_distances
    return returnVal
    

In [14]:
import random
r = random.random
l = 20000
d1 = np.array([[r(), r()] for i in range(l)])
d2 = d1 + 0.5
d3 = d2 + 0.5
drums = np.concatenate([d1, d2, d3])
labels = [0]*l + [1]*l + [2]*l

analysis = clusterGeometryMetrics(drums, labels)
print analysis['pairwise_intersect_areas']


<type 'list'>
[[0.33541703279490875, 0.08374817918763068, 0.0], [0.08374817918763068, 0.33541703279490875, 0.08374817918763067], [0.0, 0.08374817918763068, 0.33541703279490875]]
